In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from kielmat.datasets import keepcontrol
import kineticstoolkit.lab as ktk

In [20]:
recording = keepcontrol.load_recording(
    dataset_path=Path('../datasets/keepcontrol'),
    id="pp002",
    task="walkSlow",
    tracking_systems="omc"
)

In [21]:
marker_df = recording.data["omc"]
fs = recording.channels["omc"]["sampling_frequency"].iloc[0].astype(float)

In [22]:
marker_df = marker_df[[col for col in marker_df.columns if not col.endswith("_n/a")]]
marker_df

,l_toe_POS_x,l_toe_POS_y,l_toe_POS_z,l_heel_POS_x,l_heel_POS_y,l_heel_POS_z,l_ank_POS_x,l_ank_POS_y,l_ank_POS_z,l_sk1_POS_x,...,start_1_POS_z,start_2_POS_x,start_2_POS_y,start_2_POS_z,end_1_POS_x,end_1_POS_y,end_1_POS_z,end_2_POS_x,end_2_POS_y,end_2_POS_z
0,-335.448147,629.924345,25.050978,NaN,NaN,NaN,-567.186943,610.347597,78.805453,-502.076335,...,79.445464,917.002926,106.086489,74.293606,5684.326365,1889.547368,57.100769,5856.413263,883.665108,53.069631
1,-335.460796,629.956566,25.022214,NaN,NaN,NaN,-567.213027,610.344865,78.817840,-502.010473,...,79.443287,916.986116,106.080529,74.266089,5684.321887,1889.556462,57.108411,5856.375971,883.654160,53.040584
2,-335.432605,629.928490,25.054483,NaN,NaN,NaN,-567.201569,610.311672,78.785987,-502.044025,...,79.463126,916.987124,106.079844,74.307390,5684.335538,1889.539415,57.089054,5856.368554,883.596861,53.237349
3,-335.446386,630.004978,25.052327,NaN,NaN,NaN,-567.187119,610.364207,78.817328,-502.649942,...,79.453986,917.001127,106.056444,74.292980,5684.323615,1889.530640,57.083778,5856.392434,883.677634,53.082505
4,-335.589092,629.981562,25.259407,NaN,NaN,NaN,-567.660980,610.554729,79.487492,-502.677402,...,79.433717,917.283813,105.955388,74.377702,5684.325414,1889.552181,57.086501,5856.381835,883.645142,53.054584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2579,6440.865935,1652.762073,1.189836,6276.638536,1427.532651,24.649658,6272.808791,1499.678064,69.306766,6369.068134,...,79.242978,917.296819,106.475911,74.296904,5683.574573,1889.855304,56.721711,5857.575738,882.996433,52.364573
2580,6440.505553,1652.893331,1.384448,6277.978482,1426.706231,24.579138,6273.554323,1498.299183,69.213819,6368.248275,...,79.255021,917.303412,106.478539,74.296924,5683.578303,1889.859829,56.696947,5857.860836,882.909102,52.307003
2581,6440.232289,1652.845509,1.734327,6279.047374,1425.901345,24.579269,6273.648433,1497.343914,69.116306,6367.071022,...,79.266662,917.283809,106.497876,74.306678,5683.609818,1889.853468,56.700835,5858.072069,882.755723,52.171419
2582,6439.903320,1652.825898,2.160087,6279.827818,1425.315593,24.371718,6273.772808,1496.621737,69.176844,6366.057215,...,79.249168,917.287709,106.488546,74.287130,5683.605602,1889.858563,56.708471,5858.312703,882.660339,52.022640


In [23]:
markers_ts = ktk.TimeSeries(
    data={
        mrk: np.hstack((
            marker_df.loc[:, [f"{mrk}_POS_{x}" for x in "xyz"]].values / 1000,
            np.ones((marker_df.shape[0], 1))
        ))
        for mrk in [mrk[:-6] for mrk in marker_df.columns if mrk.endswith("_x")]
    }, 
    time=marker_df.index / fs, 
    data_info={
        mrk: {"Unit": "mm"}
        for mrk in [mrk[:-6] for mrk in marker_df.columns if mrk.endswith("_x")]
    },
    time_info={"Unit": "s"}
)

In [24]:
%matplotlib qt5
p = ktk.Player(markers_ts)
p.up = "z"
p.anterior = "-y"
p.play()

In [25]:
markers_ts.data.keys()

dict_keys(['l_toe', 'l_heel', 'l_ank', 'l_sk1', 'l_sk2', 'l_sk3', 'l_sk4', 'l_th1', 'l_th2', 'l_th3', 'l_th4', 'r_toe', 'r_heel', 'r_ank', 'r_sk1', 'r_sk2', 'r_sk3', 'r_sk4', 'r_th1', 'r_th2', 'r_th3', 'r_th4', 'l_asis', 'r_asis', 'l_psis', 'r_psis', 'm_ster1', 'm_ster2', 'm_ster3', 'l_sho', 'l_ua', 'l_elbl', 'l_frm', 'l_wrr', 'l_wru', 'l_hand', 'r_sho', 'r_ua', 'r_elbl', 'r_frm', 'r_wrr', 'r_wru', 'r_hand', 'lf_hd', 'rf_hd', 'lb_hd', 'rb_hd', 'start_1', 'start_2', 'end_1', 'end_2'])

In [26]:
interconnections = dict()

interconnections["LFoot"] = {
    "Color": (0.0, 0.5, 1.0),
    "Links": [
        ["l_toe", "l_heel", "l_ank", "l_toe"],
    ]
}
interconnections["RFoot"] = {
    "Color": (1.0, 0.5, 0.0),
    "Links": [
        ["r_toe", "r_heel", "r_ank", "r_toe"],
    ]
}
p.set_interconnections(interconnections)